In [1]:
from socceraction.spadl.statsbomb import convert_to_actions
from socceraction.data.statsbomb import StatsBombLoader
from socceraction.spadl.config import actiontypes, bodyparts
import socceraction.vaep.labels as lab
import socceraction.vaep.features as fs
import socceraction.spadl as spadl
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
from xgboost import XGBClassifier
import socceraction.vaep.formula as vaepformula

In [2]:
api = StatsBombLoader(root="data/statsbomb", getter="local")

In [3]:
# Competitions Input
competitions_df = api.competitions()
competitions_df[competitions_df["country_name"] == "International"]

,season_id,competition_id,competition_name,country_name,competition_gender,season_name
25,274,1470,FIFA U20 World Cup,International,male,1979
26,106,43,FIFA World Cup,International,male,2022
27,3,43,FIFA World Cup,International,male,2018
28,55,43,FIFA World Cup,International,male,1990
29,54,43,FIFA World Cup,International,male,1986
30,51,43,FIFA World Cup,International,male,1974
31,272,43,FIFA World Cup,International,male,1970
32,270,43,FIFA World Cup,International,male,1962
33,269,43,FIFA World Cup,International,male,1958
68,107,72,Women's World Cup,International,female,2023


In [4]:
# Games Input (Competition_id, Season_id)
games_world_cup_2018_df = api.games(43, 3)
games_world_cup_2018_df[games_world_cup_2018_df['game_day'] == 1]

,game_id,season_id,competition_id,competition_stage,game_day,game_date,home_team_id,away_team_id,home_score,away_score,venue,referee
9,7578,3,43,Group Stage,1,2018-06-15 14:00:00,774,783,0,1,\tEkaterinburg Arena,Björn Kuipers
12,7536,3,43,Group Stage,1,2018-06-18 17:00:00,782,798,3,0,\tOlimpiyskiy Stadion Fisht,Janny Sikazwe
15,7539,3,43,Group Stage,1,2018-06-19 17:00:00,789,787,1,2,Otkritie Bank Arena,Nawaf Abdulla Ghayyath Shukralla
16,7538,3,43,Group Stage,1,2018-06-18 14:00:00,790,791,1,0,Stadion Nizhny Novgorod (Nizhniy Novgorod),Joel Antonio Aguilar Chicas
17,7576,3,43,Group Stage,1,2018-06-15 20:00:00,780,772,3,3,\tOlimpiyskiy Stadion Fisht,Gianluca Rocchi
20,7537,3,43,Group Stage,1,2018-06-18 20:00:00,777,768,1,2,Volgograd Arena,Wilmar Alexander Roldán Pérez
25,7529,3,43,Group Stage,1,2018-06-16 21:00:00,785,775,2,0,Stadion Kaliningrad,Sandro Ricci
26,7534,3,43,Group Stage,1,2018-06-17 17:00:00,770,794,0,1,Stadion Luzhniki,Alireza Faghani
31,7530,3,43,Group Stage,1,2018-06-16 12:00:00,771,792,2,1,Ak Bars Arena,Andrés Cunha
35,7535,3,43,Group Stage,1,2018-06-17 14:00:00,795,786,0,1,\tSolidarnost Arena,Malang Diedhiou


In [5]:
# Teams Input (game_id)
teams_world_cup_2018_df = api.teams(7525)
teams_world_cup_2018_df.head()

,team_id,team_name
0,796,Russia
1,799,Saudi Arabia


In [6]:
# Player Input (game_id)
players_df = api.players(7525)
players_df

,game_id,team_id,player_id,player_name,nickname,jersey_number,is_starter,starting_position_id,starting_position_name,minutes_played
0,7525,796,5170,Mário Figueira Fernandes,Mário Fernandes,2,True,2,Right Back,96
1,7525,796,5171,Roman Zobnin,None,11,True,13,Right Center Midfield,96
2,7525,796,5172,Igor Akinfeev,None,1,True,1,Goalkeeper,96
3,7525,796,5174,Sergei Ignashevich,None,4,True,5,Left Center Back,96
4,7525,796,5175,Ilya Kutepov,None,3,True,3,Right Center Back,96
5,7525,796,5177,Aleksandr Golovin,None,17,True,16,Left Midfield,96
6,7525,796,5179,Daler Kuzyaev,None,7,False,0,Substitute,31
7,7525,796,5180,Alan Dzagoev,None,9,True,19,Center Attacking Midfield,23
8,7525,796,5181,Yury Gazinskiy,None,8,True,15,Left Center Midfield,96
9,7525,796,5182,Aleksandr Samedov,None,19,True,12,Right Midfield,65


In [7]:
# Events Input (game_id)
event_world_cup_2018_df = api.events(7525)
# event_world_cup_2018_df

In [8]:
# Convert Wyscout to SPADL Format Input (Events Dataframe, home_team_id) --> Rusia
spadl_events_wc_2018_df = convert_to_actions(event_world_cup_2018_df, 796)
spadl_events_wc_2018_df = spadl.add_names(spadl_events_wc_2018_df)
spadl_events_wc_2018_df

,game_id,original_event_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,type_id,result_id,bodypart_id,action_id,type_name,result_name,bodypart_name
0,7525,e23943da-dc40-49f3-a7a0-e269288098c3,1,0.0,799,5196.0,52.058824,33.569620,75.000000,32.708861,0,1,5,0,pass,success,foot_right
1,7525,fd89c08f-16ad-4102-a3ab-4dd8d58ae687,1,1.0,799,5173.0,75.000000,32.708861,74.117647,33.569620,21,1,0,1,dribble,success,foot
2,7525,c4ba0e20-8090-4285-945e-e2f9d136ae21,1,2.0,799,5173.0,74.117647,33.569620,26.470588,12.911392,0,0,5,2,pass,fail,foot_right
3,7525,576d4649-3b45-4de2-b9f7-692c3d53a12d,1,5.0,796,5175.0,32.647059,10.329114,38.823529,4.303797,0,0,0,3,pass,fail,foot
4,7525,a7a37529-06c3-462f-bab5-5135acfa524a,1,7.0,799,5178.0,38.823529,4.303797,25.588235,4.303797,0,1,4,4,pass,success,foot_left
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,7525,9e245d96-5c52-4622-9e20-f5dbbfa895ad,2,2867.0,796,5179.0,86.470588,25.822785,86.470588,23.240506,0,1,5,1884,pass,success,foot_right
1885,7525,93798d11-6c4a-4a09-b874-d8a14f0b51d6,2,2867.0,796,5177.0,86.470588,23.240506,87.352941,22.379747,21,1,0,1885,dribble,success,foot
1886,7525,c1950d4e-4a0d-4d6f-96dd-2401771028ac,2,2868.0,799,5185.0,87.352941,22.379747,87.352941,22.379747,8,4,0,1886,foul,yellow_card,foot
1887,7525,19a9887b-f870-4ffd-a197-b12ea9301261,2,2937.0,796,5177.0,86.470588,24.101266,105.000000,30.987342,13,1,5,1887,shot_freekick,success,foot_right


In [9]:
# 1. convert actions to game states
# home_team_id = 796 # Russia
home_team_id = 799 # Saudi Arabia
gamestates = fs.gamestates(spadl_events_wc_2018_df, 3)
gamestates = fs.play_left_to_right(gamestates, home_team_id)

In [10]:
# 2. compute features
xfns = [
    fs.actiontype, 
    fs.result, 
    fs.bodypart, 
    fs.time, 
    fs.startlocation, 
    fs.endlocation,
    fs.startpolar,
    fs.endpolar,
    fs.movement,
    fs.team,
    fs.time_delta,
    fs.space_delta,
    fs.goalscore
]
X = pd.concat([fn(gamestates) for fn in xfns], axis=1)
X

,type_id_a0,type_id_a1,type_id_a2,result_id_a0,result_id_a1,result_id_a2,bodypart_id_a0,bodypart_id_a1,bodypart_id_a2,period_id_a0,...,time_delta_2,dx_a01,dy_a01,mov_a01,dx_a02,dy_a02,mov_a02,goalscore_team,goalscore_opponent,goalscore_diff
0,0,0,0,1,1,1,0,0,0,1,...,0.0,2.294118e+01,-0.860759,2.295732e+01,22.941176,-0.860759,22.957319,0,0,0
1,21,0,0,1,1,1,0,0,0,1,...,1.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0,0,0
2,0,21,0,0,1,1,0,0,0,1,...,2.0,0.000000e+00,0.000000,0.000000e+00,0.882353,-0.860759,1.232661,0,0,0
3,0,0,21,0,0,1,0,0,0,1,...,4.0,6.176471e+00,-2.582278,6.694546e+00,-41.470588,-23.240506,47.538730,0,0,0
4,0,0,0,1,0,0,0,0,0,1,...,5.0,0.000000e+00,0.000000,0.000000e+00,-12.352941,8.607595,15.056090,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,0,0,21,1,1,1,0,0,0,2,...,3.0,-8.823529e-01,1.721519,1.934470e+00,0.882353,7.746835,7.796923,4,0,4
1885,21,0,0,1,1,1,0,0,0,2,...,1.0,0.000000e+00,0.000000,0.000000e+00,-0.882353,-0.860759,1.232661,4,0,4
1886,8,21,0,4,1,1,0,0,0,2,...,1.0,1.421085e-14,0.000000,1.421085e-14,-0.882353,0.860759,1.232661,0,4,-4
1887,13,8,21,1,4,1,0,0,0,2,...,70.0,-8.823529e-01,1.721519,1.934470e+00,-0.882353,1.721519,1.934470,4,0,4


In [11]:
# 3. compute labels
yfns = [lab.scores, lab.concedes]
Y = pd.concat([fn(spadl_events_wc_2018_df) for fn in yfns], axis=1)
Y

,scores,concedes
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
1884,True,False
1885,True,False
1886,False,True
1887,True,False


In [12]:
# 4. load or train models
Y_hat = pd.DataFrame()
models = {}
for col in list(Y.columns):
    model = XGBClassifier(objective="binary:logistic")
    model.fit(X, Y[col])
    models[col] = model

# 5. predict scoring and conceding probabilities for each game state
for col in list(Y.columns):
    Y_hat[col] = [p[1] for p in models[col].predict_proba(X)]
Y_hat

,scores,concedes
0,0.000137,0.000160
1,0.000120,0.000176
2,0.000312,0.000089
3,0.000064,0.000897
4,0.000080,0.000144
...,...,...
1884,0.988271,0.001887
1885,0.995218,0.002409
1886,0.001636,0.761862
1887,0.991975,0.010312


In [13]:
# 6. compute VAEP value
values = vaepformula.value(spadl_events_wc_2018_df, Y_hat["scores"], Y_hat["concedes"])
values

,offensive_value,defensive_value,vaep_value
0,0.000000,-0.000000,0.000000
1,-0.000017,-0.000016,-0.000033
2,0.000192,0.000087,0.000279
3,-0.000025,-0.000585,-0.000610
4,-0.000816,-0.000081,-0.000897
...,...,...,...
1884,0.003251,-0.000835,0.002417
1885,0.006948,-0.000521,0.006426
1886,-0.000772,0.233357,0.232585
1887,0.991975,-0.010312,0.981663
